In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv")
df

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,mask
0,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor...,True
1,81b5f874-9b42-4d4f-8c50-b4bab24f8aed,The Gender Gap Reloaded: Are School Characteri...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study examines the wage gender gap of yo...,True
2,f35fdfa6-a3f9-4ef3-a858-c8e2efcd2349,The High School Environment and the Gender Gap...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Despite the striking reversal of the gender g...,True
3,e5a4c6af-0854-4f7e-b088-06f03c18a01a,Locus of Control and Peer Relationships Among ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Abstract Past research has shown that locus o...,True
4,d5ff0ecd-ccf5-4f8a-a09c-182376eed4d0,Beyond the Laboratory: Evaluating the Survey E...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,"The black-white gap in achievement, as measur...",True
...,...,...,...,...,...,...,...
4150,d90a010d-8dba-4ffa-a2f1-473a45ddb8c7,Identification of potential inhibitors of SARS...,SARS-CoV-2 genome sequence,genome sequences of 2019-nCoV,genome sequences of 2019 ncov,A case of unidentified pneumonia was reported...,True
4151,a45b5269-9937-4f04-ad51-81e705d837c4,The importance of standardisation – COVID-19 C...,COVID-19 Image Data Collection,COVID-19 Image Data Collection,covid 19 image data collection,"At the end of 2019, a new coronavirus SARS-Co...",True
4152,bd180f8e-6dfa-4874-a172-957322f46867,COVID-Net: A Tailored Deep Convolutional Neura...,COVID-19 Image Data Collection,COVID-19 Image Data Collection,covid 19 image data collection,The COVID-19 pandemic continues to have a dev...,True
4153,118d2571-5938-4363-81d0-c3083d18a607,COVID-19 detection and disease progression vis...,COVID-19 Image Data Collection,COVID-19 Image Data Collection,covid 19 image data collection,The 2019 novel Coronavirus (COVID-19) has bec...,True


In [60]:
# Every publication input will be mapped into a variable numbers of chunks (split by sentence) that are less than chunk_max_len
# These can then be batched by encoding strings, then padding them
chunk_max_len = 512
data = [[]] * len(df.index) # publication id x chunks - left in string format for flexibility in encoding
gold = [[]] * len(df.index) # publication id x chunk - if label in chunk, True else False

In [61]:
for i in range(len(df.index)):
    data[i] = []
    gold[i] = []
    chunk = ''
    for s in df['text'][i].split('. '):
        new_chunk = chunk + s.strip()
        if len(new_chunk.split(' ')) > chunk_max_len:
            gold[i].append(True if df['dataset_label'][i] in chunk else False)
            data[i].append(chunk)
            chunk = s
        else:
            chunk = new_chunk
    gold[i].append(True if df['dataset_label'][i] in chunk else False)
    data[i].append(chunk)

In [76]:
print(len(data[0]), data[0])
print(gold[0])

1 ["This article investigates an important factor in student achievement-parental involvementUsing data from the National Education Longitudinal Study (NELS), we estimate a value-added education production function that includes parental effort as an inputParental effort equations are also estimated as a function of child, parent, household, and school characteristicsOur results suggest that parental effort has a strong positive effect on achievement that is large relative to the effect of school resources and is not captured by family background variablesParents appear to reduce their effort in response to increased school resources, suggesting potential ''crowding out'' of school resources."]
[True]


In [77]:
# Observe chunked samples
for i in range(len(data)):
    if len(data[i]) > 1:
        print(i, gold[i])

1 [True, False]
3 [True, False, False]
5 [True, True, False]
6 [False, True, False]
8 [True, False, False]
9 [False, True]
10 [True, True, False, False, True, False, False, False, False, False]
12 [False, True]
14 [False, True, False, False, False, False]
15 [False, True, False, False]
16 [False, True, False]
17 [False, True]
19 [False, True, False]
22 [True, False]
23 [True, True, False]
24 [True, False]
25 [False, True]
26 [True, True, False]
27 [True, False, False, False, False, False]
28 [False, False, False, True, False, False, False, False, False, False]
29 [True, False, False, False]
30 [False, False, False, True, False]
34 [True, False, False, False]
35 [True, False]
36 [False, True, False]
37 [False, False, True, False, False]
38 [False, False, False, True]
39 [True, True, False]
41 [False, False, False, False, True, False]
42 [True, False, False]
44 [True, False]
45 [False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]

In [ ]:
# Note data and gold are of the form: # publication x chunks